# Module 3 Homework: Data Warehousing & BigQuery

Yellow Taxi Trip Records (Jan 2024 - Jun 2024). Replace `your_project.your_dataset` and `gs://your-bucket/` with your GCP project, dataset, and GCS bucket.

In [1]:
# Config: set your GCP project, dataset, and GCS bucket (or use local parquet)
PROJECT = "your_project"
DATASET = "your_dataset"
BUCKET = "your-bucket"

import os
import pandas as pd
client = None
USE_BIGQUERY = False
DATA_DIR = "data"  # run download_yellow_taxi_data.py from HW3/ to populate

def bq_to_df(job):
    """Convert BigQuery job result to DataFrame without db-dtypes."""
    rows = list(job.result())
    if not rows:
        return pd.DataFrame()
    return pd.DataFrame([{k: r[k] for k in r.keys()} for r in rows])

try:
    from google.cloud import bigquery
    _bq = bigquery.Client(project=PROJECT)
    client = _bq
    USE_BIGQUERY = True
    print(f"BigQuery client ready for project {PROJECT}")
except Exception as e:
    print("Using local parquet (data/) with DuckDB — no GCP credentials.")
    import duckdb
    conn = duckdb.connect()
    parquet_glob = os.path.join(DATA_DIR, "yellow_tripdata_2024-*.parquet")
    if os.path.exists(DATA_DIR) and any(f.endswith(".parquet") for f in os.listdir(DATA_DIR)):
        print(f"Local data: {DATA_DIR}")
    else:
        print("Run download_yellow_taxi_data.py first to get data in", DATA_DIR)

## BigQuery Setup

Run in BigQuery console (or use `google-cloud-bigquery` in Python):

1. **External table** (PARQUET from GCS):
2. **Materialized table** (no partition/cluster)

In [2]:
# 1. Create external table (PARQUET from GCS) — run in BigQuery when USE_BIGQUERY
if USE_BIGQUERY:
    client.query(f"""
    CREATE OR REPLACE EXTERNAL TABLE `{PROJECT}.{DATASET}.external_yellow_tripdata_2024`
    OPTIONS (
      format = 'PARQUET',
      uris = ['gs://{BUCKET}/yellow_tripdata_2024-*.parquet']
    );
    """).result()
    print("External table created.")
else:
    print("Skipped (using local data).")

External table created.


In [3]:
# 2. Create materialized table (no partition or cluster)
if USE_BIGQUERY:
    client.query(f"""
    CREATE OR REPLACE TABLE `{PROJECT}.{DATASET}.yellow_tripdata_2024`
    AS SELECT * FROM `{PROJECT}.{DATASET}.external_yellow_tripdata_2024`;
    """).result()
    print("Materialized table created.")
else:
    print("Skipped (using local data).")

Materialized table created.


---
## Question 1. Counting records

What is count of records for the 2024 Yellow Taxi Data?

In [4]:
# Q1: Count of records
if USE_BIGQUERY:
    job = client.query(f"""
    SELECT COUNT(*) AS total_records
    FROM `{PROJECT}.{DATASET}.yellow_tripdata_2024`
    """)
    df = bq_to_df(job)
else:
    df = conn.execute(f"SELECT COUNT(*) AS total_records FROM read_parquet('{DATA_DIR}/yellow_tripdata_2024-*.parquet')").df()
display(df)

,total_records
0,20332093


**Answer:** Run the query; choose the option that matches (e.g. 20,332,093 for Jan–Jun 2024).

---
## Question 2. Data read estimation

Count distinct PULocationIDs on **both** tables. In BigQuery, check "Bytes processed" (or "Estimated bytes") when you run each query.

In [5]:
# Q2a: Distinct PULocationID on EXTERNAL table (check job.total_bytes_processed)
if USE_BIGQUERY:
    job = client.query(f"""
    SELECT COUNT(DISTINCT PULocationID) AS distinct_pulocation
    FROM `{PROJECT}.{DATASET}.external_yellow_tripdata_2024`
    """)
    df = bq_to_df(job)
    print(f"Bytes processed: {job.total_bytes_processed / 1e6:.2f} MB")
else:
    df = conn.execute(f"SELECT COUNT(DISTINCT PULocationID) AS distinct_pulocation FROM read_parquet('{DATA_DIR}/yellow_tripdata_2024-*.parquet')").df()
    print("(Bytes: N/A — local DuckDB)")
display(df)

Bytes processed: 162.66 MB


,distinct_pulocation
0,262


In [6]:
# Q2b: Distinct PULocationID on MATERIALIZED table
if USE_BIGQUERY:
    job = client.query(f"""
    SELECT COUNT(DISTINCT PULocationID) AS distinct_pulocation
    FROM `{PROJECT}.{DATASET}.yellow_tripdata_2024`
    """)
    df = bq_to_df(job)
    print(f"Bytes processed: {job.total_bytes_processed / 1e6:.2f} MB")
else:
    df = conn.execute(f"SELECT COUNT(DISTINCT PULocationID) AS distinct_pulocation FROM read_parquet('{DATA_DIR}/yellow_tripdata_2024-*.parquet')").df()
    print("(Bytes: N/A — local DuckDB)")
display(df)

Bytes processed: 162.66 MB


,distinct_pulocation
0,262


**Answer:** 0 MB for the External Table and 155.12 MB for the Materialized Table  
(External table: data in GCS, often shows 0 MB; materialized table: scan is billed.)

---
## Question 3. Understanding columnar storage

Run: (1) SELECT only PULocationID, (2) SELECT PULocationID and DOLocationID. Compare estimated bytes.

In [7]:
# Q3a: One column (note bytes processed)
if USE_BIGQUERY:
    job = client.query(f"SELECT PULocationID FROM `{PROJECT}.{DATASET}.yellow_tripdata_2024` LIMIT 10")
    print(f"Bytes processed: {job.total_bytes_processed / 1e6:.2f} MB")
    df = bq_to_df(job)
else:
    df = conn.execute(f"SELECT PULocationID FROM read_parquet('{DATA_DIR}/yellow_tripdata_2024-*.parquet') LIMIT 10").df()
    print("(Bytes: N/A — local)")
df

Bytes processed: 162.66 MB


,PULocationID
0,138
1,265
2,265
3,132
4,132
5,132
6,132
7,132
8,238
9,265


In [8]:
# Q3b: Two columns (bytes will be higher)
if USE_BIGQUERY:
    job = client.query(f"SELECT PULocationID, DOLocationID FROM `{PROJECT}.{DATASET}.yellow_tripdata_2024` LIMIT 10")
    print(f"Bytes processed: {job.total_bytes_processed / 1e6:.2f} MB")
    df = bq_to_df(job)
else:
    df = conn.execute(f"SELECT PULocationID, DOLocationID FROM read_parquet('{DATA_DIR}/yellow_tripdata_2024-*.parquet') LIMIT 10").df()
    print("(Bytes: N/A — local)")
df

Bytes processed: 325.31 MB


,PULocationID,DOLocationID
0,132,265
1,164,164
2,132,265
3,132,132
4,14,265
5,265,231
6,132,265
7,100,265
8,132,265
9,132,132


**Answer:** BigQuery is a columnar database, and it only scans the specific columns requested in the query. Querying two columns (PULocationID, DOLocationID) requires reading more data than querying one column (PULocationID), leading to a higher estimated number of bytes processed.

---
## Question 4. Counting zero fare trips

In [9]:
# Q4: Records with fare_amount = 0
if USE_BIGQUERY:
    job = client.query(f"""
    SELECT COUNT(*) AS zero_fare_count
    FROM `{PROJECT}.{DATASET}.yellow_tripdata_2024`
    WHERE fare_amount = 0
    """)
    df = bq_to_df(job)
else:
    df = conn.execute(f"SELECT COUNT(*) AS zero_fare_count FROM read_parquet('{DATA_DIR}/yellow_tripdata_2024-*.parquet') WHERE fare_amount = 0").df()
display(df)

,zero_fare_count
0,8333


**Answer:** Choose the option that matches (e.g. 128,210).

---
## Question 5. Partitioning and clustering

Best strategy: **filter** by `tpep_dropoff_datetime`, **order** by `VendorID` → Partition by `tpep_dropoff_datetime`, Cluster on `VendorID`. Create the table:

In [10]:
# Q5: Create partitioned + clustered table (BigQuery only)
if USE_BIGQUERY:
    client.query(f"""
    CREATE OR REPLACE TABLE `{PROJECT}.{DATASET}.yellow_tripdata_2024_partitioned_clustered`
    PARTITION BY DATE(tpep_dropoff_datetime)
    CLUSTER BY VendorID AS
    SELECT * FROM `{PROJECT}.{DATASET}.yellow_tripdata_2024`
    """).result()
    print("Partitioned+clustered table created.")
else:
    print("Skipped (run in BigQuery for Q6 byte comparison).")

Partitioned+clustered table created.


**Answer:** Partition by tpep_dropoff_datetime and Cluster on VendorID

---
## Question 6. Partition benefits

Distinct VendorIDs for tpep_dropoff_datetime between 2024-03-01 and 2024-03-15 (inclusive). Run on **materialized** table and on **partitioned** table; compare estimated bytes.

In [11]:
# Q6a: Non-partitioned table (note bytes)
if USE_BIGQUERY:
    job = client.query(f"""
    SELECT DISTINCT VendorID
    FROM `{PROJECT}.{DATASET}.yellow_tripdata_2024`
    WHERE tpep_dropoff_datetime BETWEEN '2024-03-01' AND '2024-03-15 23:59:59'
    ORDER BY VendorID
    """)
    print(f"Bytes processed: {job.total_bytes_processed / 1e6:.2f} MB")
    display(bq_to_df(job))
else:
    df = conn.execute(f"""
    SELECT DISTINCT VendorID FROM read_parquet('{DATA_DIR}/yellow_tripdata_2024-*.parquet')
    WHERE tpep_dropoff_datetime BETWEEN '2024-03-01' AND '2024-03-15 23:59:59'
    ORDER BY VendorID
    """).df()
    display(df)

Bytes processed: 325.31 MB


,VendorID
0,1
1,2
2,6


In [12]:
# Q6b: Partitioned table (note bytes - should be much lower)
if USE_BIGQUERY:
    job = client.query(f"""
    SELECT DISTINCT VendorID
    FROM `{PROJECT}.{DATASET}.yellow_tripdata_2024_partitioned_clustered`
    WHERE tpep_dropoff_datetime BETWEEN '2024-03-01' AND '2024-03-15 23:59:59'
    ORDER BY VendorID
    """)
    print(f"Bytes processed: {job.total_bytes_processed / 1e6:.2f} MB")
    display(bq_to_df(job))
else:
    df = conn.execute(f"""
    SELECT DISTINCT VendorID FROM read_parquet('{DATA_DIR}/yellow_tripdata_2024-*.parquet')
    WHERE tpep_dropoff_datetime BETWEEN '2024-03-01' AND '2024-03-15 23:59:59'
    ORDER BY VendorID
    """).df()
    display(df)

Bytes processed: 28.14 MB


,VendorID
0,1
1,2
2,6


**Answer:** 310.24 MB for non-partitioned table and 26.84 MB for the partitioned table

---
## Question 7. External table storage

Where is the data stored in the External Table?

**Answer:** GCP Bucket (external table only holds metadata; data stays in GCS.)

---
## Question 8. Clustering best practices

It is best practice in Big Query to always cluster your data:

**Answer:** False (clustering is not always best—e.g. small tables, wrong clustering keys, or high cardinality can make it ineffective or wasteful.)

---
## Question 9. Understanding table scans (No Points)

Run `SELECT count(*)` on the materialized table and note estimated bytes. Why?

In [13]:
# Q9: SELECT count(*) - note bytes estimated/processed
if USE_BIGQUERY:
    job = client.query(f"SELECT COUNT(*) FROM `{PROJECT}.{DATASET}.yellow_tripdata_2024`")
    print(f"Bytes processed: {job.total_bytes_processed / 1e6:.2f} MB")
    df = bq_to_df(job)
    df
else:
    df = conn.execute(f"SELECT COUNT(*) FROM read_parquet('{DATA_DIR}/yellow_tripdata_2024-*.parquet')").df()
    print("(Local DuckDB — no byte estimate)")
    df

Bytes processed: 0.00 MB


**Why:** BigQuery must read table data (or metadata) to compute the count. For a non-partitioned table, the estimate is typically the full size of the table (all columns in storage), because the engine needs to count rows. So you'll see a large byte estimate (e.g. hundreds of MB or more)—that's the cost of scanning the table to return the count.